# 📌 Coleta de Dados – Explicação Geral
Este notebook realiza a coleta de dados de jogos usando a **RAWG API**, processa os textos, limpa descrições e salva o dataset final para ser usado no modelo de classificação.


## 🔹 Importações
Nesta seção importamos todas as bibliotecas necessárias para fazer requisições HTTP,
manipular tabelas e organizar os dados.

In [19]:
import requests
import pandas as pd

API_KEY = "c2444955082141798db05daa6690e426"
print("Using RAWG API key:", API_KEY[:6] + "..." )


Using RAWG API key: c24449...


## 🔹 Funções de Coleta
Aqui definimos funções que chamam a API da RAWG, coletam jogos, filtram informações
e retornam um dataframe com descrições e gêneros.

In [20]:
def get_top_games(n_pages=3):
    """
    Busca jogos populares na RAWG API e, para cada jogo, faz uma requisição
    de detalhes para obter a descrição completa (description_raw) e os gêneros.
    Cada página retorna até 40 jogos.
    """
    all_games = []
    
    for page in range(1, n_pages + 1):
        list_url = "https://api.rawg.io/api/games"
        params = {
            "key": API_KEY,
            "page": page,
            "page_size": 40
        }

        print(f"Buscando página {page} de jogos...")
        resp = requests.get(list_url, params=params)
        data = resp.json()

        if "results" not in data:
            print("Resposta inesperada na página", page, ":", data)
            continue

        for item in data["results"]:
            game_id = item["id"]
            name = item.get("name", "")

            # Endpoint de detalhes para pegar description_raw e gêneros
            detail_url = f"https://api.rawg.io/api/games/{game_id}"
            detail_params = {"key": API_KEY}
            detail_resp = requests.get(detail_url, params=detail_params)
            detail = detail_resp.json()

            description = detail.get("description_raw", "") or ""
            genres = [g["name"] for g in detail.get("genres", [])]

            all_games.append({
                "name": name,
                "description": description,
                "genres": genres
            })

    print("Total de jogos coletados:", len(all_games))
    return all_games


In [21]:
# Coleta os dados da API
data = get_top_games(n_pages=10)

# Cria DataFrame
df = pd.DataFrame(data)
print("Shape antes da limpeza:", df.shape)
df.head()


Buscando página 1 de jogos...
Buscando página 2 de jogos...
Buscando página 3 de jogos...
Buscando página 4 de jogos...
Buscando página 5 de jogos...
Buscando página 6 de jogos...
Buscando página 7 de jogos...
Buscando página 8 de jogos...
Buscando página 9 de jogos...
Buscando página 10 de jogos...
Total de jogos coletados: 400
Shape antes da limpeza: (400, 3)


,name,description,genres
0,Grand Theft Auto V,"Rockstar Games went bigger, since their previo...",[Strategy]
1,The Witcher 3: Wild Hunt,"The third game in a series, it holds nothing b...","[Action, RPG]"
2,Portal 2,Portal 2 is a first-person puzzle game develop...,"[Shooter, Puzzle]"
3,Counter-Strike: Global Offensive,Counter-Strike is a multiplayer phenomenon in ...,[Shooter]
4,Tomb Raider (2013),A cinematic revival of the series in its actio...,[Action]


## 🔹 Limpeza e Preparação dos Dados
Nesta etapa removemos valores vazios, textos curtos, duplicatas e dados inconsistentes
para garantir melhor qualidade no treinamento do modelo.

In [22]:
# Remove descrições vazias ou muito curtas
df = df[df["description"].notna()]
df = df[df["description"].str.len() > 10]

print("Shape depois da limpeza:", df.shape)
print(df["description"].str.len().describe())

df.head()


Shape depois da limpeza: (400, 3)
count     400.00000
mean     1163.60000
std       621.03571
min       265.00000
25%       977.00000
50%      1071.00000
75%      1192.75000
max      8745.00000
Name: description, dtype: float64


,name,description,genres
0,Grand Theft Auto V,"Rockstar Games went bigger, since their previo...",[Strategy]
1,The Witcher 3: Wild Hunt,"The third game in a series, it holds nothing b...","[Action, RPG]"
2,Portal 2,Portal 2 is a first-person puzzle game develop...,"[Shooter, Puzzle]"
3,Counter-Strike: Global Offensive,Counter-Strike is a multiplayer phenomenon in ...,[Shooter]
4,Tomb Raider (2013),A cinematic revival of the series in its actio...,[Action]


## 🔹 Salvando o Resultado
O dataset final é exportado para um arquivo CSV dentro da pasta `data/`, pronto para
ser usado no notebook de treinamento.

In [23]:
# Salva o dataset expandido
output_path = "../data/games_extended.csv"
df.to_csv(output_path, index=False)
print("Arquivo salvo em:", output_path)
df.head()


Arquivo salvo em: ../data/games_extended.csv


,name,description,genres
0,Grand Theft Auto V,"Rockstar Games went bigger, since their previo...",[Strategy]
1,The Witcher 3: Wild Hunt,"The third game in a series, it holds nothing b...","[Action, RPG]"
2,Portal 2,Portal 2 is a first-person puzzle game develop...,"[Shooter, Puzzle]"
3,Counter-Strike: Global Offensive,Counter-Strike is a multiplayer phenomenon in ...,[Shooter]
4,Tomb Raider (2013),A cinematic revival of the series in its actio...,[Action]
